# Introduction to Physics Informed Neural Networks (PINNs)

In this notebook we will outline the brief theory about Physics Informed Neural Networks (PINNs).

## Neural Network Solver Methodology 

In this section we provide a brief introduction to solving differential equations with neural networks. The idea is to use a neural network to approximate the solution to the given differential equation and boundary conditions. We train this neural network by constructing a loss function for how well the neural network is satisfying the differential equation and boundary conditions. If the network is able to minimize this loss function then it will in effect, solve the given differential equation.

To illustrate this idea we will give an example of solving the following problem,

\begin{equation} \label{1d_equation}
    \mathbf{P} : \left\{\begin{matrix}
\frac{\mathrm{d}^2 u}{\mathrm{d} x^2}(x) = f(x), \\ 
\\
u(0) = u(1) = 0,
\end{matrix}\right.
\end{equation}

We start by constructing a neural network $u_{net}(x)$. The input to this network is a single value $x \in \mathbb{R}$ and its output is also a single value $u_{net}(x) \in \mathbb{R}$. We suppose that this neural network is infinitely differentiable, $u_{net} \in C^{\infty}$. The typical neural network used is a deep fully connected network where the activation functions are infinitely differentiable. 

Next we need to construct a loss function to train this neural network. We easily encode the boundary conditions as a loss in the following way:

\begin{equation}
  L_{BC} = u_{net}(0)^2 + u_{net}(1)^2
\end{equation}

For encoding the equations, we need to compute the derivatives of $u_{net}$. Using automatic differentiation we can do so and compute $\frac{\mathrm{d}^2 u_{net}}{\mathrm{d} x^2}(x)$. This allows us to write a loss function of the form:

\begin{equation} \label{sumation_loss}
  L_{residual} = \frac{1}{N}\sum^{N}_{i=0} \left( \frac{\mathrm{d}^2 u_{net}}{\mathrm{d} x^2}(x_i) - f(x_i) \right)^2
\end{equation}

Where the $x_i$'s are a batch of points sampled in the interior, $x_i \in (0, 1)$. Our total loss is then $L = L_{BC} + L_{residual}$. Optimizers such as Adam are used to train this neural network. Given $f(x)=1$, the true solution is $\frac{1}{2}(x-1)x$. Upon solving the problem, you can obtain good agreement between the exact solution and the neural network solution as shown in Figure below.

<img src="single_parabola.png" alt="Drawing" style="width: 500px;"/>

## Parmeterized Problems

One important advantage of a neural network solver over traditional numerical methods is its ability to solve parameterized geometries. To illustrate this concept we solve a parameterized version of the above problem. Suppose we want to know how the solution to this equation changes as we move the position on the boundary condition $u(l)=0$. We can parameterize this position with a variable $l \in [1,2]$ and our equation now has the form,

\begin{equation} \label{1d_equation2}
    \mathbf{P} : \left\{\begin{matrix}
\frac{\mathrm{d}^2 u}{\mathrm{d} x^2}(x) = f(x), \\ 
\\
u(0) = u(l) = 0,
\end{matrix}\right.
\end{equation}

To solve this parameterized problem we can have the neural network take $l$ as input, $u_{net}(x,l)$. The losses then take the form,

\begin{equation}
  L_{residual} = \int_1^2 \int_0^l \left( \frac{\mathrm{d}^2 u_{net}}{\mathrm{d} x^2}(x,l) - f(x) \right)^2 dx dl \approx \left(\int_1^2 \int^l_0 dxdl\right) \frac{1}{N} \sum^{N}_{i=0} \left(\frac{\mathrm{d}^2 u_{net}}{\mathrm{d} x^2}(x_i, l_i) - f(x_i)\right)^2
\end{equation}

\begin{equation}
  L_{BC} = \int_1^2 (u_{net}(0,l))^2 + (u_{net}(l,l) dl \approx \left(\int_1^2 dl\right) \frac{1}{N} \sum^{N}_{i=0} (u_{net}(0, l_i))^2 + (u_{net}(l_i, l_i))^2
\end{equation}

In figure below we see the solution to the differential equation for various $l$ values after optimizing the network on this loss. While this example problem is overly simplistic, the ability to solve parameterized geometries presents significant industrial value. Instead of performing a single simulation we can solve multiple designs at the same time and for reduced computational cost. More examples of this can be found in [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/index.html).

<img src="every_parabola.png" alt="Drawing" style="width: 500px;"/>

## Inverse Problems 

Another useful application of a neural network solver is solving inverse problems. In an inverse problem, we start with a set of observations and then use those observations to calculate the causal factors that produced them. To illustrate how to solve inverse problems with a neural network solver, we give the example of inverting out the source term $f(x)$ from same equation from above problem. Suppose we are given the solution $u_{true}(x)$ at 100 random points between 0 and 1 and we want to determine the $f(x)$ that is causing it. We can do this by making two neural networks $u_{net}(x)$ and $f_{net}(x)$ to approximate both $u(x)$ and $f(x)$. These networks are then optimized to minimize the following losses;

\begin{equation}
  L_{residual} \approx \left(\int^1_0 dx\right) \frac{1}{N} \sum^{N}_{i=0} \left(\frac{\mathrm{d}^2 u_{net}}{\mathrm{d} x^2}(x_i, l_i) - f_{net}(x_i)\right)^2
\end{equation}

\begin{equation}
  L_{data} = \frac{1}{100} \sum^{100}_{i=0} (u_{net}(x_i) - u_{true}(x_i))^2
\end{equation}

Using the function $u_{true}(x)=\frac{1}{48} (8 x (-1 + x^2) - (3 sin(4 \pi x))/\pi^2)$ the solution for $f(x)$ is $x + sin(4 \pi x)$. We solve this problem and compare the results in Figures below.

Comparison of true solution for $f(x)$ and the function approximated by the NN:
<img src="inverse_parabola.png" alt="Drawing" style="width: 500px;"/>

Comparison of $u_{net}(x)$ and the train points from $u_{true}$:
<img src="inverse_parabola_2.png" alt="Drawing" style="width: 500px;"/>

More examples of solving an inverse problem can be found in the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/index.html).


# Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).
